In [ ]:
import os
import pandas as pd
import sys
from xbbg import blp
sys.path.append('../src')
import utils

etf_ticker = 'EFA'
asof = '2025-09-25'

hold_df = pd.read_csv(f'cached_holdings/{etf_ticker}_{asof}.csv')
country = 'BRITAIN'

In [ ]:
hold_df['ratio'] = hold_df['adr_turnover'] / hold_df['turnover_usd']
country_df = hold_df[hold_df['country'] == country]
#country_df = country_df[(country_df['ratio'] < 0.8) & (country_df['adr_turnover'] > 1e6)]
country_df = country_df[(country_df['adr_turnover'] > 1e6)]
country_df['adr_turnover'].sum()
country_df[['id','adr']].to_csv(f'../data/raw/{country}_adr_list.csv', index=False)

In [26]:
country_df['adr_exchange'].value_counts()

adr_exchange
OOTC    159
XNYS      9
Name: count, dtype: int64

In [ ]:
start_date = '2025-03-01'
end_date = '2025-09-30'
adr_price_filename = f'cached_adr_prices/{country}_adr_prices_{start_date}_{end_date}.csv'
if os.path.exists(adr_price_filename):
    adr_df = pd.read_csv(adr_price_filename, index_col=0, parse_dates=True)
else:
    adr_df = blp.bdh(tickers=country_df['adr'].tolist(), flds=['PX_MID'], start_date=start_date, end_date=end_date, adjust='all').droplevel(1, axis=1)
    os.makedirs('cached_adr_prices', exist_ok=True)
    adr_df.to_csv(adr_price_filename)

In [100]:
# sofr_filename = 'cached_holdings/SOFRRATE_Index_2020-01-01_2025-09-30.csv'
# if os.path.exists(sofr_filename):
#     sofr_df = pd.read_csv(sofr_filename, index_col=0, parse_dates=True)
# else:
#     sofr_df = blp.bdh('SOFRRATE Index', 'PX_LAST', '2020-01-01', '2025-09-30').droplevel(1, axis=1)
#     sofr_df.columns = ['SOFR']
#     sofr_df.to_csv(sofr_filename)

# tonar_filename = 'cached_holdings/MUTKCALM_Index_2020-01-01_2025-09-30.csv'
# if os.path.exists(tonar_filename):
#     tonar_df = pd.read_csv(tonar_filename, index_col=0, parse_dates=True)
# else:
#     tonar_df = blp.bdh('MUTKCALM Index', 'PX_LAST', '2020-01-01', '2025-09-30').droplevel(1, axis=1)
#     tonar_df.columns = ['TONAR']
#     tonar_df.to_csv(tonar_filename)

# rates_df = pd.merge(sofr_df, tonar_df, left_index=True, right_index=True, how='inner')
# ((1 + (90/360)*rates_df['TONAR']/100).div(1 + (90/360)*rates_df['SOFR']/100)).diff().sort_values()

In [101]:
import pandas as pd
import pandas_market_calendars as mcal

start_date = '2025-03-01'
end_date = '2025-09-30'
exchanges = country_df['exchange'].unique().tolist() + ['NYSE']
cals = [mcal.get_calendar(ex) for ex in exchanges]
close_time = pd.DataFrame({ex: mcal.get_calendar(ex).schedule(start_date=start_date, end_date=end_date)['market_close'] for ex in exchanges})
close_time['XLON'] = close_time['XLON'].ffill()
close_time.index = [d.strftime('%Y-%m-%d') for d in close_time.index]

In [ ]:
active_filename = f'cached_holdings/{country}_active_contracts_{start_date}_{end_date}.csv'
if os.path.exists(active_filename):
    active_df = pd.read_csv(active_filename, index_col=0, parse_dates=True)
else:
    active_df =blp.bdh(['Z 1 Index', 'BP1 Curncy'], 'FUT_CUR_GEN_TICKER', start_date=start_date, end_date=end_date).droplevel(1,axis=1).bfill()
    active_df.to_csv(active_filename)

In [104]:
delivery_filename = f'cached_holdings/{country}_delivery_dates_{start_date}_{end_date}.csv'
if os.path.exists(delivery_filename):
    delivery_df = pd.read_csv(delivery_filename, index_col=0, parse_dates=True)
else:
    all_dfs = []
    for col in active_df.columns:
        bbg_asset_name = col.split()[-1]
        bbg_contract = (active_df[col] + ' ' + bbg_asset_name).unique().tolist()
        col_df = blp.bds(bbg_contract, 'FUT_DLV_DT_FIRST')
        all_dfs.append(col_df)

    delivery_df = pd.concat(all_dfs, axis=0)
    delivery_df.columns = ['delivery_date']
    delivery_df.to_csv(delivery_filename)

C:\Users\PeterMalonis\AppData\Local\Temp\ipykernel_36636\2899991003.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  delivery_df = pd.read_csv(delivery_filename, index_col=0, parse_dates=True)


In [105]:
index_df = pd.read_parquet('../data/raw/futures/bbo-1m/symbol=Z')
fx_df = pd.read_parquet('../data/raw/futures/bbo-1m/symbol=6B')

In [106]:
index_df['symbol'] = index_df['symbol'].str.replace('   FM','').str.replace('002','').str.replace('!','')

In [ ]:
index_active = active_df[['Z 1 Index']].rename(columns={'Z 1 Index': 'active_contract'})
for year in range(20, 26):
    year_str = str(year)
    index_active['active_contract'] = index_active['active_contract'].str.replace(year_str, year_str[1])

index_active['active_contract'] = index_active['active_contract'].str.replace('Z ', 'Z')
index_active.index = [d.strftime('%Y-%m-%d') for d in active_df.index]
index_df = index_df.merge(index_active, left_on='date', right_index=True, how='left')
index_df = index_df[index_df['symbol'] == index_df['active_contract']]

In [ ]:
index_df = index_df.merge(close_time, left_on='date', right_index=True, how='left')
index_df['mid'] = (index_df['bid_px_00'] + index_df['ask_px_00']) / 2

foreign_close_df = index_df[index_df.index <= index_df['XLON']]
us_close_df = index_df[index_df.index <= index_df['NYSE']]
foreign_close_df = foreign_close_df.groupby('date').last()['mid'].rename('foreign_close')
us_close_df = us_close_df.groupby('date').last()['mid'].rename('US_close')
merged_index_df = pd.merge(foreign_close_df, us_close_df, left_index=True, right_index=True, how='inner')

In [ ]:
index_active

In [109]:
fx_active = active_df[['BP1 Curncy']].rename(columns={'BP1 Curncy': 'active_contract'})
for year in range(20, 26):
    year_str = str(year)
    fx_active['active_contract'] = fx_active['active_contract'].str.replace(year_str, year_str[1])

fx_active['active_contract'] = fx_active['active_contract'].str.replace('BP', '6B')
fx_active.index = [d.strftime('%Y-%m-%d') for d in active_df.index]
fx_df = fx_df.merge(fx_active, left_on='date', right_index=True, how='left')

fx_df = fx_df[fx_df['symbol'] == fx_df['active_contract']]
fx_df = fx_df.merge(close_time, left_on='date', right_index=True, how='left')
fx_df['mid'] = (fx_df['bid_px_00'] + fx_df['ask_px_00']) / 2

foreign_close_df = fx_df[fx_df.index <= fx_df['XLON']]
us_close_df = fx_df[fx_df.index <= fx_df['NYSE']]
foreign_close_df = foreign_close_df.groupby('date').last()['mid'].rename('foreign_close')
us_close_df = us_close_df.groupby('date').last()['mid'].rename('US_close')
merged_fx_df = pd.merge(foreign_close_df, us_close_df, left_index=True, right_index=True, how='inner')

In [110]:
fx_ret = merged_fx_df['US_close'] / merged_fx_df['foreign_close'] - 1
index_ret = merged_index_df['US_close'] / merged_index_df['foreign_close'] - 1
index_ret_usd = (1+fx_ret) * (1+index_ret) - 1

In [111]:
adr_ret = adr_df.pct_change().shift(-1).iloc[1:]
index_ret_usd.index = pd.to_datetime(index_ret_usd.index)
adr_ret.index = pd.to_datetime(adr_ret.index)
all_df = pd.merge(adr_ret, index_ret_usd.rename('index_return'), left_index=True, right_index=True, how='inner')
all_df = all_df.dropna()
X = all_df[['index_return']].values
Y = all_df.drop(columns=['index_return']).values

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# N_train = 250
N_test = 30
X_train = X[:-N_test]
Y_train = Y[:-N_test]
X_test = X[-N_test:]
Y_test = Y[-N_test:]

model = LinearRegression()
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)
r2_score(Y_test, Y_pred, multioutput='raw_values')

NameError: name 'X' is not defined